![Crowded city](city-1265055_1280.jpg)

In the quest for efficiency and effectiveness in urban transportation, finding the optimal routes to take passengers from their initial locations to their desired destinations is paramount. This challenge is not just about reducing travel time; it's about enhancing the overall experience for both drivers and passengers, ensuring safety, and minimizing environmental impact. 

You have been asked to revolutionize the way taxis navigate the urban landscape, ensuring passengers reach their destinations swiftly, safely, and satisfactorily. As an initial step, your goal is to build a reinforcement learning agent that solves this problem within a simulated environment.

## The Taxi-v3 environment
The Taxi-v3 environment is a strategic simulation, offering a grid-based arena where a taxi navigates to address daily challenges akin to those faced by a taxi driver. This environment is defined by a 5x5 grid where the taxi's mission involves picking up a passenger from one of four specific locations (marked as Red, Green, Yellow, and Blue) and dropping them off at another designated spot. The goal is to accomplish this with minimal time on the road to maximize rewards, emphasizing the need for route optimization and efficient decision-making for passenger pickup and dropoff.

### Key Components:
- **Action Space:** Comprises six actions where 0 moves the taxi south, 1 north, 2 east, 3 west, 4 picks up a passenger, and 5 drops off a passenger.
- **Observation Space:** Comprises 500 discrete states, accounting for 25 taxi positions, 5 potential passenger locations, and 4 destinations. 
- **Rewards System:** Includes a penalty of -1 for each step taken without other rewards, +20 for successful passenger delivery, and -10 for illegal pickup or dropoff actions. Actions resulting in no operation, like hitting a wall, also incur a time step penalty.

![Taxi-v3 environment snapshot](Taxi_snap.png)


In [1]:
# Re-run this cell to install and import the necessary libraries and load the required variables
!pip install gymnasium[toy_text] imageio
import numpy as np
import gymnasium as gym
import imageio
from IPython.display import Image
from gymnasium.utils import seeding
import matplotlib.pyplot as plt

# Initialize the Taxi-v3 environment
env = gym.make("Taxi-v3", render_mode='rgb_array')

# Seed the environment for reproducibility
env.np_random, _ = seeding.np_random(42)
env.action_space.seed(42)
np.random.seed(42)

# Maximum number of actions per training episode
max_actions = 100 
num_episodes = 100

# Frames
frames = []

# Alpha and Gamma
learning_rate = 0.9
gamma = 0.9

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 138.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 82.9 MB/s eta 0:00:00


In [2]:
num_states = env.observation_space.n
num_actions = env.action_space.n
print("\nNumber of States: ", num_states)
print("\nNumber of Actions: ", num_actions)
policy = {state: action for state in range(num_states) for action in range(num_actions)}
print(env.unwrapped.P[498])
#Q Table
q_table =np.zeros((num_states, num_actions))


Number of States:  500

Number of Actions:  6
{0: [(1.0, 498, -1, False)], 1: [(1.0, 398, -1, False)], 2: [(1.0, 498, -1, False)], 3: [(1.0, 478, -1, False)], 4: [(1.0, 498, -10, False)], 5: [(1.0, 498, -10, False)]}


In [ ]:
def epsilon_greedy(state):
    if rand <epsilon:
        return env.action_space.sample()
    else:
        np.argmax(q_table[state])

In [3]:
def update_q_table(state, action, reward, next_state):
    q_table[state, action] = (1 - learning_rate) * q_table[state, action] + learning_rate * (reward + gamma * np.max(q_table[next_state]))

episode_total_reward = np.zeros(num_episodes)
save_frequency = 100 

for i in range(num_episodes):
    terminated = False
    steps = 0
    episode_returns = 0 
    state, info = env.reset(seed=42)
    
    while (not terminated and steps <= max_actions):
        action = epsilon_greedy(state)
        next_state, reward, terminated, truncated, info = env.step(action)
        steps += 1
        update_q_table(state, action, reward, next_state)
        state = next_state
        episode_returns += reward
    
    episode_total_reward[i] = episode_returns
    epsilon= max(min_epsilon, epsilon*epsilon_decay)
    
    # Optionally print progress
    if (i + 1) % 100 == 0:
        print(f"Episode {i+1}/{num_episodes}, Average reward: {np.mean(episode_total_reward[max(0, i-99):i+1]):.2f}")

print("Average reward per random episode: ", np.mean(episode_total_reward))
print("\nLearned Q table: \n", q_table)

Episode 100/100, Average reward: -387.56
Average reward per random episode:  -387.56

Learned Q table: 
 [[  0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.    ]
 [ -0.99    -0.9     -0.9     -1.728   -0.99   -10.629 ]
 ...
 [  0.       0.       0.       0.       0.       0.    ]
 [  0.      -1.8648  -0.99     0.      -9.9      0.    ]
 [  0.       0.       0.       0.       0.       0.    ]]


In [4]:
for i in range(16): # Execute maximum 16 moves
    action = policy[state] 
    state, reward, terminated, truncated, info = env.step(action)
    episode_total_reward += reward
    frames.append(env.render())
    if terminated:
      break    

In [5]:
# Once you are done, run this cell to visualize the agent's behavior through the episode
# Save frames as a GIF
import imageio
from IPython.display import Image, display

imageio.mimsave('taxi_agent_behavior.gif', frames, fps=5)

# Display GIF
gif_path = "taxi_agent_behavior.gif" 
display(Image(filename=gif_path))

Error: Output too large to be added to notebook